<a href="https://colab.research.google.com/github/rohitd3/Deutsch-Jozsa-QKD-HRB/blob/main/GITHUB_ROHIT_CirqFunction_HRB_QDKDJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade matplotlib imgaug plotnine mizani albumentations cirq

# the Pandas mismatch hasn't been a problem, because we aren't using it in our code
#   ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 1.1.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 12.0 MB/s 
     |████████████████████████████████| 948 kB 44.9 MB/s 
     |████████████████████████████████| 4.7 MB 32.0 MB/s 
     |████████████████████████████████| 63 kB 314 kB/s 
     |████████████████████████████████| 116 kB 27.3 MB/s 
     |████████████████████████████████| 944 kB 47.3 MB/s 
     |████████████████████████████████| 9.8 MB 48.2 MB/s 
     |████████████████████████████████| 48.3 MB 94 kB/s 
     |████████████████████████████████| 57 kB 3.3 MB/s 
     |████████████████████████████████| 576 kB 44.7 MB/s 
     |████████████████████████████████| 594 kB 63.4 MB/s 
     |████████████████████████████████| 1.8 MB 49.5 MB/s 
     |████████████████████████████████| 66 kB 5.1 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 220 kB 48.9 MB/s 
     |████████████████████████████████|

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

In [1]:
"""
    This file contains the Cirq simulation of few variable sized
    DJ-packets with support for two different basis. This forms the
    proof of concept building block for the various HRB schemes.

    :copyright: (c) 2020-2022 by Rohit De.
    :license: MIT License, see LICENSE.txt for more details.
"""

import cirq
import numpy as np
from time import process_time, monotonic
import subprocess
from matplotlib import pyplot
from scipy.stats import *
import statistics

USE_MULTI_BASIS = True
#USE_MULTI_BASIS = False
#--------------------------------------------------------------------------

def BLOCH_SPHERE_COMPARE_STATES(result, items):
  for index in range(items):
    state = cirq.bloch_vector_from_state_vector(result.final_state_vector,index)
    print("DJ Qubit",index, "| x=", np.around(state[0], 4), " y=", np.around(state[1], 4)," z=", np.around(state[2], 4))

#--------------------------------------------------------------------------

#Eve measures the expected Qubits in the intercepted DJ-packet
def EVE(eve_intercepted_DJpacket,eve_expected_DJsize):
    EveMeasure=cirq.Circuit()
    
    # Eve measures all qubits in DJ packet, including target
    for i in range(eve_expected_DJsize):
      EveMeasure.append(cirq.measure(eve_intercepted_DJpacket[i]))  
    
    yield EveMeasure

#------------------------------------------------------------------------------------------

def ALICE_SENDING(DJpacket,DJsize):
    target=DJpacket[DJsize-1]
    # equal superposition over input bits
    hadmardseq=cirq.Circuit()
    
    for i in range(DJsize-1):
      hadmardseq.append(cirq.H(DJpacket[i]))
    
    # phase kickback trick
    yield cirq.X(target), cirq.H(target)
    yield hadmardseq 
   
    if USE_MULTI_BASIS == True:
      if DJsize == 3:
        yield [cirq.ry(np.pi/2).on(DJpacket[1]),cirq.ry(np.pi/2).on(DJpacket[2])] 
      elif  DJsize == 5:
        yield [cirq.ry(np.pi/2).on(DJpacket[0]),cirq.ry(np.pi/2).on(DJpacket[2])]
      else:
        yield [cirq.ry(np.pi/2).on(DJpacket[0]),cirq.ry(np.pi/2).on(DJpacket[1])]
#------------------------------------------------------------------------------------------

def ALICE_RECEIVING(DJpacket,DJsize):    
    
    if USE_MULTI_BASIS == True:
      if DJsize == 3:
        yield [cirq.ry(-np.pi/2).on(DJpacket[1]),cirq.ry(-np.pi/2).on(DJpacket[2])] 
      elif  DJsize == 5:
        yield [cirq.ry(-np.pi/2).on(DJpacket[0]),cirq.ry(-np.pi/2).on(DJpacket[2])]
      else:
        yield [cirq.ry(-np.pi/2).on(DJpacket[0]),cirq.ry(-np.pi/2).on(DJpacket[1])]

    target=DJpacket[DJsize-1]
    hadmardseq=cirq.Circuit()
    
    for i in range(DJsize-1):
      hadmardseq.append(cirq.H(DJpacket[i]))
    
    paulixseq=cirq.Circuit()
    
    for i in range(DJsize-1):
      paulixseq.append(cirq.X(DJpacket[i]))
    
    yield hadmardseq, cirq.H(target)
    yield paulixseq,cirq.X(target).controlled_by(*DJpacket[:-1])
    yield cirq.measure(target)

#------------------------------------------------------------------------------------------

def BOB_BALANCED_Uf(DJpacket,DJsize): 
    if USE_MULTI_BASIS == True:
      if DJsize == 3:
        yield [cirq.ry(-np.pi/2).on(DJpacket[1]),cirq.ry(-np.pi/2).on(DJpacket[2])] 
      elif  DJsize == 5:
        yield [cirq.ry(-np.pi/2).on(DJpacket[0]),cirq.ry(-np.pi/2).on(DJpacket[2])]
      else:
        yield [cirq.ry(-np.pi/2).on(DJpacket[0]),cirq.ry(-np.pi/2).on(DJpacket[1])]

    target=DJpacket[DJsize-1]
    #creating balanced oracle 'notx' 
    bal_cir = cirq.Circuit()
    
    for i in range(DJsize-1):
      bal_cir.append(cirq.CNOT(DJpacket[i],target))
    
    bal_cir.append(cirq.X(target))
    yield bal_cir

    if USE_MULTI_BASIS == True:
      if DJsize == 3:
        yield [cirq.ry(np.pi/2).on(DJpacket[1]),cirq.ry(np.pi/2).on(DJpacket[2])] 
      elif  DJsize == 5:
        yield [cirq.ry(np.pi/2).on(DJpacket[0]),cirq.ry(np.pi/2).on(DJpacket[2])]
      else:
        yield [cirq.ry(np.pi/2).on(DJpacket[0]),cirq.ry(np.pi/2).on(DJpacket[1])]


#------------------------------------------------------------------------------------------


def BOB_CONSTANT_Uf(DJpacket,DJsize):    
    
    if USE_MULTI_BASIS == True:
      if DJsize == 3:
        yield [cirq.ry(-np.pi/2).on(DJpacket[1]),cirq.ry(-np.pi/2).on(DJpacket[2])] 
      elif  DJsize == 5:
        yield [cirq.ry(-np.pi/2).on(DJpacket[0]),cirq.ry(-np.pi/2).on(DJpacket[2])]
      else:
        yield [cirq.ry(-np.pi/2).on(DJpacket[0]),cirq.ry(-np.pi/2).on(DJpacket[1])]
    
    target=DJpacket[DJsize-1]
    #creating constant oracle 'x' 
    const_circuit = cirq.Circuit()
    const_circuit.append(cirq.X(target))
    yield const_circuit

    if USE_MULTI_BASIS == True:
      if DJsize == 3:
        yield [cirq.ry(np.pi/2).on(DJpacket[1]),cirq.ry(np.pi/2).on(DJpacket[2])] 
      elif  DJsize == 5:
        yield [cirq.ry(np.pi/2).on(DJpacket[0]),cirq.ry(np.pi/2).on(DJpacket[2])]
      else:
        yield [cirq.ry(np.pi/2).on(DJpacket[0]),cirq.ry(np.pi/2).on(DJpacket[1])]


#++++++++++++++++++++++++++++++++++++++++++++++++++
#++++++++++++++++++++++++++++++++++++++++++++++++++


# Hopping with 2 different sized DJ-packet (size 3 and 5) with target reordering and use of different bases B0/B1
# (0, B0), (T, B1), (1, B1)
# (0, B1), (1, B0), (2, B1), (T, B0), (3, B0)

# Hopping with 2 different sized DJ-packet (size 3 and 5) and use of different bases B0/B1
# (0, B0), (1, B1), (T, B1)
# (0, B1), (1, B0), (2, B1), (3, B0), (T, B0)

#qubitprotocol=[3,5]
#qubitprotocol=[4,6]
#qubitprotocol=[6]

# 22 DJ-packets
qubitprotocol=[4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,6]

'''
# 50 X 2 = 100 DJ-packets 
qubitprotocol=[4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5]


# 50 X 10 = 500 DJ-packets 
qubitprotocol=[4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,
               4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5]

'''






print("\n ################################################ PROGRAM START ################################################################### \n")
ALICE_SIMULATOR = cirq.Simulator()
BOB_SIMULATOR = cirq.Simulator()
EVE_SIMULATOR = cirq.Simulator()

#t1_start = process_time()
for ii in range(1):
  for numqbits in qubitprotocol:
    #print('\n\n ================starting for new numqbits============================ :', numqbits)

    qubits=[cirq.LineQubit(i) for i in range(numqbits)] #assigning qubits
    #target=qubits[numqbits-1] #assigning the target qubit

    #print("\n\n\n ############################################# Start QUANTUM KD SIMULATION USING DJ ALGO for DJ packet size = ", numqbits, "##########################\n")

    orderofqubit=[]

    for q in range(numqbits):
        orderofqubit.append(qubits[q])
   
    print("\n\n =================================  ALICE SENDING START ============================== \n")
    # result1 = ALICE_SIMULATOR.simulate(cirq.Circuit(ALICE_SENDING(qubits,numqbits)),qubit_order=orderofqubit,initial_state=0)
    Alice_send_circuit = cirq.Circuit(ALICE_SENDING(qubits,numqbits))
    #print(Alice_send_circuit)
    result1 = ALICE_SIMULATOR.simulate(Alice_send_circuit,qubit_order=orderofqubit,initial_state=0)    
    #print('\n (STEP 1) ALICE SENDING REQUEST TO BOB With DJ-Packet State: {}'.format(result1))
    #print(result1.final_state_vector)
    BLOCH_SPHERE_COMPARE_STATES(result1, numqbits)
    print("\n =================================  ALICE SENDING END ============================== \n\n")


   

    print("\n\n @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  BOB START @@@@@@@@@@@@@@@@@@@@@@@@@@@@ \n")
    Bob_recieve_circuit = cirq.Circuit(BOB_BALANCED_Uf(qubits,numqbits))
    #print(Bob_recieve_circuit)
    result2 = BOB_SIMULATOR.simulate(Bob_recieve_circuit,qubit_order=orderofqubit,initial_state=result1.final_state_vector)
    #result2 = BOB_SIMULATOR.simulate(cirq.Circuit(BOB_CONSTANT_Uf(qubits,numqbits)),qubit_order=orderofqubit,initial_state=result1.final_state_vector)

    #print('\n (STEP 2) BOB getting DJ-packet Request from ALICE, applying Oracle, and SENDING BACK TO ALICE DJ-packet state: {}' .format(result2)) 
    #print(result2.final_state_vector)
    BLOCH_SPHERE_COMPARE_STATES(result2, numqbits)
    print("\n @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  BOB END @@@@@@@@@@@@@@@@@@@@@@@@@@@@ \n\n")
   
 
    ##########EEEEEEEEEEEEEEVVVVVVVVVVVVVVEEEEEEEEEEEEEEEEEEEEE###############
    if USE_MULTI_BASIS == True:
      print("\n\n *************************  EVE START *******************************\n")
      Eve_recieve_circuit = cirq.Circuit(EVE(qubits,numqbits))
      #print(Eve_recieve_circuit)
      result2 = EVE_SIMULATOR.simulate(Eve_recieve_circuit,qubit_order=orderofqubit,initial_state=result2.final_state_vector)
      print("\n **** PASSIVE LISTENING BETWEEN BOB --> ALICE): EVE INTERCEPTING DJ-packet *** ".format(result2))
      
      #print(result2.final_state_vector)
      BLOCH_SPHERE_COMPARE_STATES(result2, numqbits)
      print("\n *************************  EVE END *******************************\n\n")
    ###########EEEEEEEEEEEEEEVVVVVVVVVVVVVVVEEEEEEEEEEEEEEEEEEEE##############

   
    print("\n\n +++++++++++++++++++++++++++++  ALICE RECEIVING START ++++++++++++++++++++++++++++ \n")
    Alice_receiving_circuit = cirq.Circuit(ALICE_RECEIVING(qubits,numqbits))
    #print(Alice_receiving_circuit)
    FINAL_RESULT = ALICE_SIMULATOR.simulate(Alice_receiving_circuit,qubit_order=orderofqubit,initial_state=result2.final_state_vector)
    #print('\n (STEP 3) ALICE GOT BACK DJ-packet FROM BOB Measures to Determine Uf type (constant/Balanced) Bob applied: {}' .format(FINAL_RESULT)) 
    #print(FINAL_RESULT.final_state_vector)
    BLOCH_SPHERE_COMPARE_STATES(FINAL_RESULT, numqbits)
    print("\n +++++++++++++++++++++++++++++  ALICE RECEIVING END ++++++++++++++++++++++++++++ \n\n")
 

#t1_stop = process_time()
#TimeTaken = t1_stop - t1_start
#print('TimeTaken = ', TimeTaken)
    
print("\n ################################################ PROGRAM END ################################################################### \n")





 ################################################ PROGRAM START ################################################################### 



 =================================  ALICE SENDING START ============================== 

DJ Qubit 0 | x= 0.0  y= 0.0  z= -1.0
DJ Qubit 1 | x= 0.0  y= 0.0  z= -1.0
DJ Qubit 2 | x= 1.0  y= 0.0  z= 0.0
DJ Qubit 3 | x= -1.0  y= 0.0  z= 0.0

 =================================  ALICE SENDING END ============================== 




 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  BOB START @@@@@@@@@@@@@@@@@@@@@@@@@@@@ 

DJ Qubit 0 | x= 0.0  y= 0.0  z= 1.0
DJ Qubit 1 | x= 0.0  y= 0.0  z= 1.0
DJ Qubit 2 | x= -1.0  y= 0.0  z= 0.0
DJ Qubit 3 | x= -1.0  y= 0.0  z= 0.0

 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  BOB END @@@@@@@@@@@@@@@@@@@@@@@@@@@@ 




 *************************  EVE START *******************************


 **** PASSIVE LISTENING BETWEEN BOB --> ALICE): EVE INTERCEPTING DJ-packet *** 
DJ Qubit 0 | x= 0.0  y= 0.0  z= 1.0
DJ Qubit 1 | x= 0.0  y= 0.0  z= 1.0
DJ Qubit 2